# 1. import packages 导入包

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from dataclasses import dataclass

torch.manual_seed(1024)


# 2. Define parameters for GPT 定义GPT的一些参数

In [3]:
@dataclass
class GPTConfig:
    block_size: int = 512 #文本最大长度 max sequence
    batch_size: int = 12
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    hidden_dim: int = n_embd
    dropout: float = 0.1
    head_size: int = n_embd // n_head
    # follow gpt2 tokenzier
    vocab_size: int = 50257


# 3. Define the Structure of GPT

In [5]:
# 1. single head attention
class SingleHeadAttention(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.key = nn.Linear(config.hidden_dim, config.head_size)
        self.query = nn.Linear(config.hidden_dim, config.head_size)
        #attention_mask通过register_buffer注册
        #不用计算梯度，节约内存显寸，速度更快
        self.register_buff(
            "attention_mask",
            # tril 下三角
            # block_size 512
            torch.tril(
                torch.ones(config.block_size, config.block_size)
            )
        )
        self.dropout = nn.Dropout(config.dropout)